In [32]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [33]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/train_data/train.csv', header=None)

In [34]:
df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)

In [35]:
df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [36]:
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [37]:
df.head()

,label,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [48]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(data, percentage_of_sentences=None):
    
    train_sentences = data['text']
    y_train = data['label']
    
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
    X_train = [text_to_word_sequence(_) for _ in train_sentences]
  
    return X_train, y_train

In [49]:
def get_data(X):
    tk = Tokenizer()
    tk.fit_on_texts(X)
    vocab_size = len(tk.word_index)
    print(f'There are {vocab_size} different words in your corpus')
    X_token = tk.texts_to_sequences(X)

    X_pad = pad_sequences(X_token, dtype='float32', padding='post')
    
    return X_pad, vocab_size

In [50]:
X_train, y_train = load_data(df, percentage_of_sentences=10)

In [51]:
X_train_pad, vocab_size = get_data(X_train)

There are 68901 different words in your corpus


In [43]:
print("X_pad.shape", X_train_pad.shape)

X_pad.shape (56000, 1036)


In [46]:
from tensorflow.keras import layers, Sequential

# Size of your embedding space = size of the vector representing each word
embedding_size = 100

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # 16 +1 for the 0 padding
    input_length=1036, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer :)
))

model.add(layers.LSTM(20))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1036, 100)         6890200   
                                                                 
 lstm_2 (LSTM)               (None, 20)                9680      
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 6,899,901
Trainable params: 6,899,901
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train_pad, y_train, epochs=5, batch_size=16, verbose=1)

Epoch 1/5
3500/3500 [==============================] - 580s 164ms/step - loss: -18.5593 - accuracy: 0.5398
Epoch 2/5
3500/3500 [==============================] - 637s 182ms/step - loss: -51.1889 - accuracy: 0.5399
Epoch 3/5
1147/3500 [========>.....................] - ETA: 8:45 - loss: -72.8604 - accuracy: 0.5395

KeyboardInterrupt: 